In [1]:
import pickle

# pre-process initial params from NonRigid ICP registration results
param_folder = '/home/yan/Data2/3D_Body_Reconstruction/Dataset/scans/Optimized_Registered_NOMO3D_Dataset1/Original_A_Posed/parameter_male/'
param_file = param_folder + 'male_0024.pkl'

with open(param_file, 'rb') as f:
    param_data = pickle.load(f)

init_betas = param_data[72:]
print init_betas.shape
init_pose = param_data[:72]  # in radian
print init_pose.shape

(10,)
(72,)


In [2]:
## SMPL model
root_folder = '/home/yan/Data2/NOMO_Project_Summary/MVA/MVA_rebuttal_code/Shape_Parameter_Optimization/'

import sys
sys.path.append(root_folder+'SMPL/SMPL_python_v.1.0.0/smpl/')
from smpl_webuser.serialization import load_model
import numpy as np
import math
from scipy.sparse import csc_matrix


smpl_male_model = root_folder + 'SMPL/SMPL_python_v.1.0.0/smpl/models/basicModel_m_lbs_10_207_0_v1.0.0.pkl'
model = load_model(smpl_male_model)
# zeros pose
model.pose[:] = np.zeros(72)
model.betas[:] = np.zeros(10)

print "Zero SMPL Loaded"
#-----------------------------------------------------------------------------------------
Faces = model.f + 1              # faces, => .obj starts from index 1
T0 = model.r                     # vertices for zero pose + zeros shape, N x 3 => (6890,3)
W = model.weights                # a set of blend weights W,   N x K => (6890, 24)
Bs = model.shapedirs             # Shape blend shapes # (6890, 3, 10)
Bp = model.posedirs              # Pose blend shapes  # (6890, 3, 207)
tree_table = model.kintree_table # Joints Tree, 
J_regressor = model.J_regressor  # 24 * 6890, a function to predict K joint location
J_regressor = csc_matrix(J_regressor, dtype=np.float64).toarray()

Zero SMPL Loaded


In [3]:
# 0 is the root, K = 23
def findAncestor(kk):
    ancestorSet = []
    parent = tree_table[0, kk]
    while parent < 24:
        ancestorSet.append(parent)
        parent = tree_table[0, parent]
        
    return np.array(ancestorSet)

# find Joints Locations, betas => (10,)
def getJoints(betas):
    cur_betas = np.expand_dims(betas, axis=1)       # (10, 1)
    shape_replacement = np.matmul(Bs, cur_betas)    # (6890, 3, 1)
    shape_replacement = shape_replacement[:, :, 0]  # (6890, 3)
    transformedV = T0 + shape_replacement           # (6890, 3)
    Joints = np.matmul(J_regressor, transformedV)   # (24, 3)
    return Joints

# R = 23 * 9 = 207, relative rotation matrices
def getRotationMatrix(theta):
    R = []
    for w in theta:
        w_norm = np.linalg.norm(w)
        if w_norm == 0:
            w_mean = w
        else:
            w_mean = w / w_norm
        skew_sym_w = skew_sym_matrix(w_mean)
        I = np.identity(3)
        exp_w = I + skew_sym_w * math.sin(w_norm) + np.linalg.matrix_power(skew_sym_w, 2) * math.cos(w_norm)
        exp_w = np.reshape(exp_w, (9,))
        R.append(exp_w[0, :])
    R = np.reshape(R, (207, 1))
    return R

# return the skew symmetric matrix of the 3-vector w
def skew_sym_matrix(w):
    skew_sym_w = np.zeros((3,3))
    skew_sym_w[0, 1] = -w[2]
    skew_sym_w[0, 2] = w[1]
    skew_sym_w[1, 0] = w[2]
    skew_sym_w[1, 2] = -w[0]
    skew_sym_w[2, 0] = -w[1]
    skew_sym_w[2, 1] = w[0]
    
    return np.matrix(skew_sym_w)

# seems there are some problems with vector norm
def GkMatrix(theta, J, k):
    GK = np.identity(4)
    Ak = findAncestor(k) # the ordered set of joint ancestors of joint k
    for j in Ak:
        J_j = J[j, :]
        w_j = theta[j, :]
        w_j_norm = np.linalg.norm(w_j)
        if w_j_norm == 0:
            w_j_mean = w_j # the unit norm axis of rotation
        else:
            w_j_mean = w_j / w_j_norm # the unit norm axis of rotation
            
        skew_sym_w_j = skew_sym_matrix(w_j_mean)
        
        # rotation matrix using the Rodrigues formula
        I = np.identity(3)
        exp_w_j = I + skew_sym_w_j * math.sin(w_j_norm) + np.linalg.matrix_power(skew_sym_w_j, 2) * math.cos(w_j_norm)
        
        # production
        item = np.zeros((4,4))
        item[0:3, 0:3] = exp_w_j
        item[0:3, 3] = np.transpose(J_j)
        item[3, 3] = 1
        
        GK = np.matmul(GK, item)
        
    return GK

In [4]:
# G matrix
theta = np.reshape(init_pose, (24,3))  # in radian
theta_star = np.reshape(np.zeros(72), (24, 3))  # for the rest pose (zero-pose)

# calculate the Joints locations
cur_betas = init_betas
Joints = getJoints(cur_betas)

# Shape blend shape
BS_replacement = np.matmul(Bs, cur_betas)

# relative rotation matrices
R_theta = getRotationMatrix(theta[1:, :])  # except the root
R_theta_star = getRotationMatrix(theta_star[1:, :])
# R = R_theta - R_theta_star

# Pose blend shape replacement 
# BP_replacement = np.matmul(Bp, R)
# BP_replacement = BP_replacement[:, :, 0]
BP_replacement = np.zeros((6890, 3))
for n in range(207):
    BP_replacement = BP_replacement + (R_theta[n] - R_theta_star[n]) * Bp[:, :, n]

In [5]:
# Transformed Vertices
T_ = T0 + BS_replacement + BP_replacement
T_transformed = np.zeros((6890, 4)) # [x, y, z, w]

K = 24
for i in range(6890):
    t_i = np.reshape(np.array([T_[i, 0], T_[i, 1], T_[i, 2], 1]), (4,1)) # [x, y, z, w]
    t_i = t_i.astype(float)
    t_transformed = np.reshape(T_transformed[i, :], (4,1))
    
    for k in range(K):
        GK = GkMatrix(theta, Joints, k)
        GK_star = GkMatrix(theta_star, Joints, k)
        GK_prime = np.matmul(GK, np.linalg.inv(GK_star))
        t_transformed = t_transformed + W[i, k] * np.matmul(GK_prime, t_i) # (4,1)
        
    T_transformed[i, :] = np.reshape(t_transformed, (4,))

In [6]:
print T_transformed[1:3, :]

[[ 0.03626883  0.32942891 -0.05727708  1.        ]
 [ 0.04137275  0.3338191  -0.06056139  1.        ]]


In [7]:
V = T_transformed[:, 0:3]

with open( 'manually_mesh.obj', 'w') as fp:
    for v in V: # [m]
        fp.write( 'v %f %f %f\n' % ( v[0], v[1], v[2]) )
    for f in Faces: # Faces are 1-based, not 0-based in obj files
        fp.write( 'f %d %d %d\n' %  (f[0], f[1], f[2]) )
        
print 'done'

done


In [8]:
# model.betas[:] = init_betas

# deg2rad = 1.0 / 180 * math.pi
# model.pose[48:60] = np.multiply(deg2rad, [-12, 2, -69, -12,-2, 69, 16, -18, 4, 16, 18, -4])
# model.pose[3:18] = np.multiply(deg2rad, [1, 0, 9, 1, 0, -8, 4, 4, 0, -3, -2, -2, -3, 2, 2])

In [9]:
# def minDist(shape_beta):
# kdtree_t = scipy.spatial.cKDTree(targetV, leafsize=6890)

# def minDist(shape_pose):
#     model.betas[:] = shape_pose
# #     model.betas[:] = shape_pose[27:]
#     print model.betas[:]
# #     model.pose[48:60] = shape_pose[:12]
# #     model.pose[3:18] = shape_pose[12:27]
    
#     modelV = model.r
#     modelV = modelV - np.mean(modelV, 0)
# #     minY = np.min(modelV, 0)[1]
# #     modelV = modelV - [0, minY, 0]

#     # KNN searching for the nearest correspondence on Target
#     # from model to target
#     kdtree_t = scipy.spatial.cKDTree(targetV, leafsize=6890)
#     dist_t, t_idx = kdtree_t.query(modelV, k=1)
#     # from target to model
#     kdtree_m = scipy.spatial.cKDTree(modelV, leafsize=6890)
#     dist_m, m_idx = kdtree_m.query(targetV, k=1)
    
# #     dist = (np.mean(dist_t) + np.mean(dist_m))/2 * 1000
#     dist = np.mean(dist_t) * 1000
    
# #     fig = plt.figure()
# #     ax = Axes3D(fig)
# #     ax.scatter(modelV[:,0], modelV[:, 1], modelV[:, 2], 'b.')
# #     ax.scatter(targetV[t_idx,0], targetV[t_idx, 1], targetV[t_idx, 2], 'r.')
# #     plt.show()
    
# #     plt.figure()
# #     plt.scatter(modelV[m_idx,0], modelV[m_idx, 1], c='b')
# #     plt.show()
    
# #     plt.figure()
# #     plt.scatter(targetV[t_idx,0], targetV[t_idx, 1], c='r')
# #     plt.show()
    
# #     print np.mean(dist_t)*1000, np.mean(dist_m)*1000, dist
#     print dist
    
#     return dist

In [10]:
# from scipy.optimize import minimize
# import scipy

# deg2rad = 1.0 / 180 * math.pi
# model.pose[48:60] = np.multiply(deg2rad, [-12, 2, -69, -12,-2, 69, 16, -18, 4, 16, 18, -4])
# model.pose[3:18] = np.multiply(deg2rad, [1, 0, 9, 1, 0, -8, 4, 4, 0, -3, -2, -2, -3, 2, 2])

# # bnds = ( (-20*deg2rad, -5*deg2rad), (-5*deg2rad,  5*deg2rad), (-90*deg2rad, -30*deg2rad), 
# #         (-20*deg2rad, -5*deg2rad), (-5*deg2rad,  5*deg2rad), (30*deg2rad,   90*deg2rad),
# #         (5*deg2rad,   30*deg2rad), (-30*deg2rad,-5*deg2rad), (0*deg2rad,    10*deg2rad),  
# #         (5*deg2rad,   30*deg2rad), (5*deg2rad,  30*deg2rad), (-10*deg2rad,   0*deg2rad),  
# #         (-5*deg2rad,   5*deg2rad), (-5*deg2rad,  5*deg2rad), (0*deg2rad,    15*deg2rad), 
# #         (-5*deg2rad,   5*deg2rad), (-5*deg2rad,  5*deg2rad), (-15*deg2rad,   0*deg2rad),
# #         (0*deg2rad,   10*deg2rad), (0*deg2rad,  10*deg2rad), (-5*deg2rad,    5*deg2rad),
# #         (-10*deg2rad,  0*deg2rad), (-10*deg2rad, 0*deg2rad), (-10*deg2rad,   0*deg2rad), 
# #         (-10*deg2rad,  0*deg2rad), (0*deg2rad,  10*deg2rad), (0*deg2rad,    10*deg2rad),)

# bnds = ((-3, 3), (-3, 3), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5))

# # res = minimize(minDist, init_betas, method='L-BFGS-B', bounds=bnds, options={'gtol': 1e-3, 'disp': True})

# # res = minimize(minDist, init_betas, method='Nelder-Mead', options={'disp': True}) # no bnds, bad

# # res = minimize(minDist, init_betas, method='Powell', options={'disp': True}) # no bnds, very bad

# res = minimize(minDist, init_betas, method='CG', options={'gtol': 1e-3, 'disp': True}) # no bnds constrains

# # res = minimize(minDist, init_betas, method='BFGS', bounds=bnds, options={'gtol': 1e-3, 'disp': True})
# # res = minimize(minDist, init_betas, method='TNC', bounds=bnds, options={'gtol': 1e-3, 'disp': True})
# # res = minimize(minDist, init_betas, method='COBYLA', bounds=bnds, options={'gtol': 1e-3, 'disp': True})
# # res = minimize(minDist, init_betas, method='SLSQP', bounds=bnds, options={'gtol': 1e-3, 'disp': True})
# # res = minimize(minDist, init_betas, method='trust-constr', bounds=bnds, options={'gtol': 1e-3, 'disp': True})
# # res = minimize(minDist, init_betas, method='dogleg', bounds=bnds, options={'gtol': 1e-3, 'disp': True})
# # res = minimize(minDist, init_betas, method='trust-ncg', bounds=bnds, options={'gtol': 1e-3, 'disp': True})
# # res = minimize(minDist, init_betas, method='trust-exact', bounds=bnds, options={'gtol': 1e-3, 'disp': True})
# # res = minimize(minDist, init_betas, method='trust-krylov', bounds=bnds, options={'gtol': 1e-3, 'disp': True})

# # res = minimize(minDist, init_param, method='L-BFGS-B', bounds=bnds, options={'gtol': 1e-5, 'disp': True})
# # x, f, d = scipy.optimize.fmin_l_bfgs_b(minDist, init_param, bounds=bnds, pgtol=1e-5)
# print res

In [11]:
# # optimal_beta = res.x[72:]
# # optimal_pose = res.x[:72]
# optimal_beta = res.x
# print optimal_beta

In [12]:
# # model.pose[:] = np.zeros(72)
# # model.betas[:] = np.zeros(10)

# deg2rad = 1.0 / 180 * math.pi
# model.pose[48:60] = np.multiply(deg2rad, [-12, 2, -69, -12,-2, 69, 16, -18, 4, 16, 18, -4])
# model.pose[3:18] = np.multiply(deg2rad, [1, 0, 9, 1, 0, -8, 4, 4, 0, -3, -2, -2, -3, 2, 2])
# # model.pose[:] = optimal_pose
# model.betas[:] = optimal_beta

# modelV = model.r - np.mean(model.r, 0)
# # save
# with open( 'CG-optimized_mesh.obj', 'w') as fp:
#     for v in modelV: # [m]
#         fp.write( 'v %f %f %f\n' % ( v[0], v[1], v[2]) )
#     for f in model.f+1: # Faces are 1-based, not 0-based in obj files
#         fp.write( 'f %d %d %d\n' %  (f[0], f[1], f[2]) )
        
# print 'done'

In [13]:
# # save
# with open( outmesh_path, 'w') as fp:
#     for v in model.r: # [m]
#         fp.write( 'v %f %f %f\n' % ( v[0], v[1], v[2]) )
#     for f in model.f+1: # Faces are 1-based, not 0-based in obj files
#         fp.write( 'f %d %d %d\n' %  (f[0], f[1], f[2]) )
        
# print 'done'

In [14]:
# T0 = model.r
# F = model.f + 1 # Faces are 1-based, not 0-based in obj files

# # initial pose 
# theta = model.pose
# deg2rad = 1.0 / 180 * math.pi
# theta[48:60] = np.multiply(deg2rad, [-12, 2, -69, -12,-2, 69, 16, -18, 4, 16, 18, -4])
# theta[3:18] = np.multiply(deg2rad, [1, 0, 9, 1, 0, -8, 4, 4, 0, -3, -2, -2, -3, 2, 2])

# J = model.J_regressor # J, the regression matrix, to estimate the joint locations, 24*6890
# W = model.weights     # W, the blend weight matrix , representing how much the rotation of part k effects the vertex i

# Sn = model.shapedirs  # Sn represents orthonormal principle components of shape displacements, 6890*3*10
# Bp = model.posedirs   # pose displacements, 6890*3*207 
# # G, a function of theta and J(beta)